In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
import lightgbm as lgb
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import joblib
from numpy import mean
from numpy import std
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score



# Data anaylisis


Data input

In [2]:
eval_data = np.load('/content/drive/MyDrive/code/python/vici_holdings_test (1)/Test/eval_data.npy')

eval_labels = np.load('/content/drive/MyDrive/code/python/vici_holdings_test (1)/Test/eval_labels.npy')

train_data = np.load('/content/drive/MyDrive/code/python/vici_holdings_test (1)/Test/train_data.npy')

train_labels = np.load('/content/drive/MyDrive/code/python/vici_holdings_test (1)/Test/train_labels.npy')

In [3]:
df_eval_data = pd.DataFrame(eval_data)
df_eval_labels = pd.DataFrame(eval_labels)
df_train_data = pd.DataFrame(train_data)
df_train_labels = pd.DataFrame(train_labels)
df_train_labels.rename(columns = {0:'Label'}, inplace = True)
# df_train_labels

Correlation map, heat map

In [ ]:
histtotal = df_train_data.hist(bins=1000000, figsize=(50,50))

In [8]:
Total_frames = [df_train_data, df_train_labels]

Total_result = pd.concat(Total_frames, axis=1, join='inner')
#specify size of heatmap
fig, ax = plt.subplots(figsize=(50, 50))

corr = Total_result.corr()


sb.heatmap(corr, cmap="Blues", annot=True, linewidths=.3)

In [9]:

# calculate the correlation matrix on the numeric columns
corr = Total_result.select_dtypes('number').corr()

# plot the heatmap
sb.heatmap(corr)

# Neural Networks

Model

In [5]:

# get the model
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X_train, X_val, X_test, y_train, y_val, y_test):
    results = list()
    n_inputs, n_outputs = X_train.shape[1], y_train.shape[1]
    # define evaluation procedure
    # cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    # enumerate folds
    # for train_ix, test_ix in cv.split(X):
    # # prepare data
    #   X_train, X_test = X[train_ix], X[test_ix]
    #   y_train, y_test = y[train_ix], y[test_ix]
    # define model
    model = get_model(n_inputs, n_outputs)
    # fit model
    model.fit(X_train, y_train, verbose=0, epochs=100)
    # make a prediction on the test set




    yhat = model.predict(X_test)
    # round probabilities to class labels
    yhat = yhat.round()
    print(yhat)
    # calculate accuracy
    acc = accuracy_score(y_test, yhat)
    pre = precision_score(y_test, yhat, average='macro')
    f1 = f1_score(y_test, yhat, average='macro')
    recall = recall_score(y_test, yhat, average='macro')

    # store result
    print('>%.3f' % acc)
    print(acc, pre, f1, recall)
    results.append(acc)
    return results



Data input, change data format

In [6]:
eval_data = np.load('/content/drive/MyDrive/code/python/vici_holdings_test (1)/Test/eval_data.npy')

eval_labels = np.load('/content/drive/MyDrive/code/python/vici_holdings_test (1)/Test/eval_labels.npy')

train_data = np.load('/content/drive/MyDrive/code/python/vici_holdings_test (1)/Test/train_data.npy')

train_labels = np.load('/content/drive/MyDrive/code/python/vici_holdings_test (1)/Test/train_labels.npy')

In [7]:
df_eval_data = pd.DataFrame(eval_data)
df_eval_labels = pd.DataFrame(eval_labels)
df_train_data = pd.DataFrame(train_data)
df_train_labels = pd.DataFrame(train_labels)
df_train_labels.rename(columns = {0:'Label'}, inplace = True)
# df_train_labels
number = 1000000
Total = 1174461

In [8]:
df_eval_data

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,-0.163385,1.728609,0.142649,-0.176952,0.117582,-0.152395,0.367092,0.396664,-0.568763,-0.659308,...,0.0,0.000000,0.000000,1.960784,0.000000,0.000000,0.336591,0.264297,0.315789,0.026261
1,-0.055390,1.519337,0.142649,-0.176952,0.117582,-0.152395,0.625994,-0.385464,-1.628485,0.106094,...,0.0,0.000000,0.000000,1.960784,0.000000,0.000000,0.336591,0.264297,0.315789,0.026261
2,-0.163385,1.519337,0.142649,-0.176952,0.117582,-0.152395,0.423988,0.171544,-0.373376,0.901865,...,0.0,0.000000,0.000000,1.960784,0.000000,0.000000,0.336591,0.264297,0.315789,0.026261
3,0.096820,-0.029401,0.142649,-0.176952,0.117582,-0.152395,0.900423,-0.511266,0.159667,-0.132701,...,0.0,0.000000,0.000000,1.960784,0.000000,0.000000,0.336591,0.264297,0.315789,0.026261
4,0.096820,1.150617,0.142649,-0.176952,0.117582,-0.152395,-2.559886,0.032492,-0.380696,0.576335,...,0.0,0.000000,0.000000,1.960784,0.000000,0.000000,0.336591,0.264297,0.315789,0.026261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1175297,-1.003403,1.080189,0.132555,-0.162858,0.133346,-0.165023,0.332277,-0.141980,-1.691026,0.816738,...,0.0,-0.554529,0.554529,4.136691,0.554529,-0.554529,0.123831,0.243421,0.363363,0.020290
1175298,-1.208349,0.028363,0.132555,-0.162858,0.133346,-0.165023,0.332277,-1.206222,-1.121051,-0.289549,...,0.0,-0.554529,0.554529,4.136691,0.554529,-0.554529,0.123831,0.243421,0.363363,0.020290
1175299,-1.078144,0.028363,0.132555,-0.162858,0.133346,-0.165023,-0.502565,-0.688454,-1.423030,0.056849,...,0.0,-0.554529,0.554529,4.136691,0.554529,-0.554529,0.123831,0.243421,0.363363,0.020290
1175300,-0.445393,0.028363,0.132555,-0.162858,0.133346,-0.165023,0.300544,-1.076358,-0.998571,0.375565,...,0.0,-0.554529,0.554529,4.136691,0.554529,-0.554529,0.123831,0.243421,0.363363,0.020290


In [9]:

testing_Total = 1175302
testing_feature = df_eval_data.to_numpy()
print(testing_feature, len(testing_feature))

print('-----------------------------------------------------')

testing_labels = df_eval_labels.to_numpy()
testing_labels = testing_labels.reshape(testing_Total, 1)
print(testing_labels, len(testing_labels))


[[-0.16338479  1.7286092   0.14264851 ...  0.26429675  0.31578947
   0.0262605 ]
 [-0.05539012  1.51933705  0.14264851 ...  0.26429675  0.31578947
   0.0262605 ]
 [-0.16338479  1.51933705  0.14264851 ...  0.26429675  0.31578947
   0.0262605 ]
 ...
 [-1.07814393  0.0283631   0.13255495 ...  0.24342105  0.36336336
   0.02028986]
 [-0.44539324  0.0283631   0.13255495 ...  0.24342105  0.36336336
   0.02028986]
 [-1.55934112  0.23100588  0.13255495 ...  0.24342105  0.36336336
   0.02028986]] 1175302
-----------------------------------------------------
[[1]
 [1]
 [1]
 ...
 [2]
 [2]
 [0]] 1175302


In [10]:
frames = [df_train_data, df_train_labels]

result = pd.concat(frames, axis=1, join='inner')

# result

In [11]:
result_shuffle = result.sample(frac = 1)

In [12]:
number = 10000
Total = 1174461
traing_data = result_shuffle[:][0:number]

validation_data = result_shuffle[:][number:]

In [13]:
training_target = traing_data['Label']
training_target = training_target.to_numpy()
training_target = training_target.reshape(number, 1)
print(training_target, len(training_target))

print('-----------------------------------------------------')

validation_target = validation_data['Label']
validation_target = validation_target.to_numpy()
validation_target = validation_target.reshape(Total - number, 1)
print(validation_target, len(validation_target))


[[1]
 [1]
 [0]
 ...
 [1]
 [0]
 [0]] 10000
-----------------------------------------------------
[[0]
 [2]
 [0]
 ...
 [1]
 [0]
 [2]] 1164461


In [14]:
traing_data=traing_data.drop(columns=['Label'])
training_feature = traing_data.to_numpy()
print(training_feature, len(training_feature))

print('-----------------------------------------------------')

validation_data=validation_data.drop(columns=['Label'])
validation_feature = validation_data.to_numpy()
print(validation_feature, len(validation_feature))


[[ 0.65243043  0.28435094  0.10842589 ...  0.30464072  0.39312039
   0.04198062]
 [ 1.03143406  0.05277634 -0.04897992 ...  0.24707602  0.36982249
   0.03883495]
 [-1.06403993  0.02688464  0.11791738 ...  0.25804039  0.4
   0.03225806]
 ...
 [ 0.53331918 -1.37426918  0.12022814 ...  0.25624082  0.31805158
   0.03751234]
 [ 1.52948289  0.00820838  1.08562183 ...  0.26188558  0.36615385
   0.04912664]
 [ 1.30905452  0.00227546 -0.00404649 ...  0.28112764  0.37325905
   0.02287582]] 10000
-----------------------------------------------------
[[ 0.8561261   0.01608569  0.16045116 ...  0.21801287  0.4
   0.04753199]
 [-0.74582357 -1.52784167  0.12886284 ...  0.28665689  0.35549872
   0.02877698]
 [ 1.98516009 -1.05871545  0.11514951 ...  0.27202797  0.42416452
   0.06532181]
 ...
 [ 1.42920911 -0.01016329  0.10322239 ...  0.27641654  0.36842105
   0.02539683]
 [-1.44527888 -3.22690894  0.09779961 ...  0.26851852  0.34482759
   0.03059072]
 [-1.45467848  0.00496695 -0.27600116 ...  0.2948421

evaluate model

In [15]:
# evaluate model
results = evaluate_model(training_feature, validation_feature, testing_feature, training_target, validation_target, testing_labels)
# summarize performance
print('Accuracy: %.3f (%.3f)' % (mean(results), std(results)))

36729/36729 [==============================] - 57s 2ms/step
[[0.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


>0.284
0.2842460916428288 0.2541921161575273 0.21970041401132945 0.28500674481004035
Accuracy: 0.284 (0.000)


# LGB - 1, with Full data

random shuffel data


In [13]:
frames = [df_train_data, df_train_labels]

result = pd.concat(frames, axis=1, join='inner')

# result

In [14]:
result_shuffle = result.sample(frac = 1)

# result_shuffle

### 分割 training data， 剩下當 validation data

In [15]:
traing_data = result_shuffle[:][0:1000000]

validation_data = result_shuffle[:][1000000:]

In [ ]:
training_target = traing_data['Label']
training_target = training_target.to_numpy()
print(training_target, len(training_target))

print('-----------------------------------------------------')

validation_target = validation_data['Label']
validation_target = validation_target.to_numpy()
print(validation_target, len(validation_target))

In [17]:
traing_data=traing_data.drop(columns=['Label'])
training_feature = traing_data.to_numpy()
print(training_feature, len(training_feature))

print('-----------------------------------------------------')

validation_data=validation_data.drop(columns=['Label'])
validation_feature = validation_data.to_numpy()
print(validation_feature, len(validation_feature))


[[-1.17252238  0.07161286 -5.91760905 ...  0.26072874  0.32919255
   0.02628697]
 [-1.95989552  0.40085257  0.71440571 ...  0.24707602  0.36982249
   0.03883495]
 [ 0.20210611 -0.01409301  0.12908552 ...  0.21541787  0.33444816
   0.03581267]
 ...
 [-0.54163439 -0.02565414  0.01037488 ...  0.23503965  0.39877301
   0.04618285]
 [-0.59689864  1.508268    0.13650141 ...  0.25055432  0.40117994
   0.06804734]
 [ 0.72277808  0.36238449  0.11238841 ...  0.29484219  0.36553525
   0.05349345]] 1000000
-----------------------------------------------------
[[ 1.66151168  1.66234459  0.16452029 ...  0.25613276  0.42535211
   0.0514064 ]
 [-0.20845227 -0.04672668 -3.82864952 ...  0.27427184  0.37463127
   0.0367893 ]
 [-0.646982    0.90994059  0.13172762 ...  0.27295492  0.40366972
   0.02525832]
 ...
 [-1.27448417  0.016716    0.02015329 ...  0.23424125  0.31893688
   0.02337398]
 [ 1.3991809  -1.39268905  0.11214464 ...  0.29039464  0.36153846
   0.03567681]
 [-0.60969255  0.0674231   0.1336989

In [19]:
# 转换为Dataset数据格式
train_data = lgb.Dataset(training_feature, label=training_target)
validation_data = lgb.Dataset(validation_feature, label=validation_target)

# 参数
params = {
    'learning_rate': 0.1,
    'lambda_l1': 0.1,
    'lambda_l2': 0.2,
    'max_depth': 50,
    'objective': 'multiclass',  # 目标函数
    'num_class': 3,
    'num_leaves':50,

}

# 模型训练
gbm = lgb.train(params, train_data, valid_sets=[validation_data])


# 模型预测
y_pred = gbm.predict(validation_feature)
y_pred = [list(x).index(max(x)) for x in y_pred]
print(y_pred)

# 模型评估
print('validation data')
print("accuracy_score: ", accuracy_score(validation_target, y_pred))

print('---------------------------------------------------------------------------------------------------------')
print("precision_score(micro): ", precision_score(validation_target, y_pred, average='micro'))
print("precision_score(macro): ", precision_score(validation_target, y_pred, average='macro'))


print('---------------------------------------------------------------------------------------------------------')
print("f1_score(micro): ", f1_score(validation_target, y_pred, average='micro'))
print("f1_score(macro): ", f1_score(validation_target, y_pred, average='macro'))


print('---------------------------------------------------------------------------------------------------------')
print("recall_score(micro): ", recall_score(validation_target, y_pred, average='micro'))
print("recall_score(macro): ", recall_score(validation_target, y_pred, average='macro'))






[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.930592 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16302
[LightGBM] [Info] Number of data points in the train set: 1000000, number of used features: 70
[LightGBM] [Info] Start training from score -0.611191
[LightGBM] [Info] Start training from score -1.520265
[LightGBM] [Info] Start training from score -1.432791
[1, 2, 0, 1, 2, 0, 2, 0, 0, 0, 2, 0, 1, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 2, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 1, 2, 0, 1, 2, 1, 0, 0, 0, 0, 1, 0, 1, 0, 2, 0, 0, 0, 0, 1, 0, 2, 2, 2, 1, 0, 1, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 1, 0, 2, 0, 0, 0, 2, 1, 2, 0, 0, 0, 0, 1, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1, 2, 0, 1, 1, 0, 2, 0, 2, 0, 1, 2, 1, 1, 0, 2, 1, 0, 0, 1, 0, 0, 0, 2, 0, 1, 1, 1, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 2, 2, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0,

In [20]:
# testing
# 模型预测
y_pred = gbm.predict(eval_data)
y_pred = [list(x).index(max(x)) for x in y_pred]
print(y_pred)

# 模型评估
print('testing data')
print("accuracy_score: ", accuracy_score(eval_labels, y_pred))

print('---------------------------------------------------------------------------------------------------------')
print("precision_score(micro): ", precision_score(eval_labels, y_pred, average='micro'))
print("precision_score(macro): ", precision_score(eval_labels, y_pred, average='macro'))


print('---------------------------------------------------------------------------------------------------------')
print("f1_score(micro): ", f1_score(eval_labels, y_pred, average='micro'))
print("f1_score(macro): ", f1_score(eval_labels, y_pred, average='macro'))


print('---------------------------------------------------------------------------------------------------------')
print("recall_score(micro): ", recall_score(eval_labels, y_pred, average='micro'))
print("recall_score(macro): ", recall_score(eval_labels, y_pred, average='macro'))



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



accuracy_score:  0.5990902763715198
---------------------------------------------------------------------------------------------------------
precision_score(micro):  0.5990902763715198
precision_score(macro):  0.5913438038339849
---------------------------------------------------------------------------------------------------------
f1_score(micro):  0.5990902763715198
f1_score(macro):  0.5587770185424975
---------------------------------------------------------------------------------------------------------
recall_score(micro):  0.5990902763715198
recall_score(macro):  0.5413977781679049


### results (1)

In [20]:


# ---------------------------------------------------------------------------------------------------------------------------------------------------
# 10萬當training data 剩下當 validation data

# 1.
# params = {
#     'learning_rate': 0.01,
#     'lambda_l1': 0.1,
#     'lambda_l2': 0.2,
#     'max_depth': 15,
#     'objective': 'multiclass',  # 目标函数
#     'num_class': 3,
#     'num_leaves':50
# }
# validation
# accuracy_score:  0.5667064695693934
# ---------------------------------------------------------------------------------------------------------
# precision_score(micro):  0.5667064695693934
# precision_score(macro):  0.6231944335821192
# ---------------------------------------------------------------------------------------------------------
# f1_score(micro):  0.5667064695693934
# f1_score(macro):  0.36088451092160473
# ---------------------------------------------------------------------------------------------------------
# recall_score(micro):  0.5667064695693934
# recall_score(macro):  0.39151925896995365


# 2.
# params = {
#     'learning_rate': 0.1,
#     'lambda_l1': 0.1,
#     'lambda_l2': 0.4,
#     'max_depth': 15,
#     'objective': 'multiclass',  # 目标函数
#     'num_class': 3,
#     'num_leaves':50
# }

# accuracy_score:  0.6061653238228284
# ---------------------------------------------------------------------------------------------------------
# precision_score(micro):  0.6061653238228284
# precision_score(macro):  0.5958161947121864
# ---------------------------------------------------------------------------------------------------------
# f1_score(micro):  0.6061653238228284
# f1_score(macro):  0.5732655082167322
# ---------------------------------------------------------------------------------------------------------
# recall_score(micro):  0.6061653238228284
# recall_score(macro):  0.5590904065660289

# accuracy_score:  0.5902278733465952
# ---------------------------------------------------------------------------------------------------------
# precision_score(micro):  0.5902278733465952
# precision_score(macro):  0.58078156076661
# ---------------------------------------------------------------------------------------------------------
# f1_score(micro):  0.5902278733465952
# f1_score(macro):  0.5487070796671593
# ---------------------------------------------------------------------------------------------------------
# recall_score(micro):  0.5902278733465952
# recall_score(macro):  0.5318457970041724


# 3.
# params = {
#     'learning_rate': 0.1,
#     'lambda_l1': 0.1,
#     'lambda_l2': 0.4,
#     'max_depth': 15,
#     'objective': 'multiclass',  # 目标函数
#     'num_class': 3,
#     'num_leaves':40
# }

# accuracy_score:  0.6061001748783809
# ---------------------------------------------------------------------------------------------------------
# precision_score(micro):  0.6061001748783809
# precision_score(macro):  0.5957669773845289
# ---------------------------------------------------------------------------------------------------------
# f1_score(micro):  0.6061001748783809
# f1_score(macro):  0.572449164319668
# ---------------------------------------------------------------------------------------------------------
# recall_score(micro):  0.6061001748783809
# recall_score(macro):  0.5578840024300376

# accuracy_score:  0.5910421321498645
# ---------------------------------------------------------------------------------------------------------
# precision_score(micro):  0.5910421321498645
# precision_score(macro):  0.5811442638710181
# ---------------------------------------------------------------------------------------------------------
# f1_score(micro):  0.5910421321498645
# f1_score(macro):  0.5515482809166886
# ---------------------------------------------------------------------------------------------------------
# recall_score(micro):  0.5910421321498645
# recall_score(macro):  0.5353378246162945


# 4.
# params = {
#     'learning_rate': 0.1,
#     'lambda_l1': 0.4,
#     'lambda_l2': 0.4,
#     'max_depth': 15,
#     'objective': 'multiclass',  # 目标函数
#     'num_class': 3,
#     'num_leaves':50
# }
# accuracy_score:  0.6060527092188549
# ---------------------------------------------------------------------------------------------------------
# precision_score(micro):  0.6060527092188549
# precision_score(macro):  0.5957786836609272
# ---------------------------------------------------------------------------------------------------------
# f1_score(micro):  0.6060527092188549
# f1_score(macro):  0.5725448584646013
# ---------------------------------------------------------------------------------------------------------
# recall_score(micro):  0.6060527092188549
# recall_score(macro):  0.5580874608470229

# accuracy_score:  0.5898866844436579
# ---------------------------------------------------------------------------------------------------------
# precision_score(micro):  0.5898866844436579
# precision_score(macro):  0.5789167030400387
# ---------------------------------------------------------------------------------------------------------
# f1_score(micro):  0.5898866844436579
# f1_score(macro):  0.5559882483599369
# ---------------------------------------------------------------------------------------------------------
# recall_score(micro):  0.5898866844436579
# recall_score(macro):  0.5422492715533687

# 5.
# params = {
#     'learning_rate': 0.1,
#     'lambda_l1': 0.1,
#     'lambda_l2': 0.4,
#     'max_depth': 20,
#     'objective': 'multiclass',  # 目标函数
#     'num_class': 3,
#     'num_leaves':50
# }

# accuracy_score:  0.6063002752077553
# ---------------------------------------------------------------------------------------------------------
# precision_score(micro):  0.6063002752077553
# precision_score(macro):  0.5962624328683996
# ---------------------------------------------------------------------------------------------------------
# f1_score(micro):  0.6063002752077553
# f1_score(macro):  0.5723641868096928
# ---------------------------------------------------------------------------------------------------------
# recall_score(micro):  0.6063002752077553
# recall_score(macro):  0.5577062406194885

# accuracy_score:  0.5911961351210157
# ---------------------------------------------------------------------------------------------------------
# precision_score(micro):  0.5911961351210157
# precision_score(macro):  0.5810823295193166
# ---------------------------------------------------------------------------------------------------------
# f1_score(micro):  0.5911961351210157
# f1_score(macro):  0.5533130748159119
# ---------------------------------------------------------------------------------------------------------
# recall_score(micro):  0.5911961351210157
# recall_score(macro):  0.537575841516982

# 6.
# params = {
#     'learning_rate': 0.1,
#     'lambda_l1': 0.1,
#     'lambda_l2': 0.4,
#     'max_depth': 50,
#     'objective': 'multiclass',  # 目标函数
#     'num_class': 3,
#     'num_leaves':50
# }
# accuracy_score:  0.6066269506291992
# ---------------------------------------------------------------------------------------------------------
# precision_score(micro):  0.6066269506291992
# precision_score(macro):  0.5947097809297143
# ---------------------------------------------------------------------------------------------------------
# f1_score(micro):  0.6066269506291992
# f1_score(macro):  0.5778122071167165
# ---------------------------------------------------------------------------------------------------------
# recall_score(micro):  0.6066269506291992
# recall_score(macro):  0.5660352798409186

# accuracy_score:  0.590686478879471
# ---------------------------------------------------------------------------------------------------------
# precision_score(micro):  0.590686478879471
# precision_score(macro):  0.5785369586987223
# ---------------------------------------------------------------------------------------------------------
# f1_score(micro):  0.590686478879471
# f1_score(macro):  0.5622374040425479
# ---------------------------------------------------------------------------------------------------------
# recall_score(micro):  0.590686478879471
# recall_score(macro):  0.5509134099630881


# ---------------------------------------------------------------------------------------------------------------------------------------------------
# 100萬當training data 剩下當 validation data

# 1.
# params = {
#     'learning_rate': 0.05,
#     'lambda_l1': 0.1,
#     'lambda_l2': 0.2,
#     'max_depth': 15,
#     'objective': 'multiclass',  # 目标函数
#     'num_class': 3,
#     'num_leaves':50
# }
#
# validation
# accuracy_score:  0.6142461638990949
# ---------------------------------------------------------------------------------------------------------
# precision_score(micro):  0.6142461638990949
# precision_score(macro):  0.6056860967384147
# ---------------------------------------------------------------------------------------------------------
# f1_score(micro):  0.6142461638990949
# f1_score(macro):  0.5790389010045467
# ---------------------------------------------------------------------------------------------------------
# recall_score(micro):  0.6142461638990949
# recall_score(macro):  0.5630457396036773
#
# testing
# accuracy_score:  0.59564860776209
# ---------------------------------------------------------------------------------------------------------
# precision_score(micro):  0.59564860776209
# precision_score(macro):  0.587362882948638
# ---------------------------------------------------------------------------------------------------------
# f1_score(micro):  0.59564860776209
# f1_score(macro):  0.5544549372177028
# ---------------------------------------------------------------------------------------------------------
# recall_score(micro):  0.59564860776209
# recall_score(macro):  0.5371439145223521



# 2.
# params = {
#     'learning_rate': 0.1,
#     'lambda_l1': 0.1,
#     'lambda_l2': 0.2,
#     'max_depth': 15,
#     'objective': 'multiclass',  # 目标函数
#     'num_class': 3,
#     'num_leaves':50
# }

# validation data
# accuracy_score:  0.6183788927038134
# ---------------------------------------------------------------------------------------------------------
# precision_score(micro):  0.6183788927038134
# precision_score(macro):  0.6094485817254246
# ---------------------------------------------------------------------------------------------------------
# f1_score(micro):  0.6183788927038134
# f1_score(macro):  0.5882431102933882
# ---------------------------------------------------------------------------------------------------------
# recall_score(micro):  0.6183788927038134
# recall_score(macro):  0.5743711936107871

# testing data
# accuracy_score:  0.5994246585133013
# ---------------------------------------------------------------------------------------------------------
# precision_score(micro):  0.5994246585133013
# precision_score(macro):  0.5918253280202577
# ---------------------------------------------------------------------------------------------------------
# f1_score(micro):  0.5994246585133013
# f1_score(macro):  0.5588596391673378
# ---------------------------------------------------------------------------------------------------------
# recall_score(micro):  0.5994246585133013
# recall_score(macro):  0.5413379608029126


# 3.
# params = {
#     'learning_rate': 0.1,
#     'lambda_l1': 0.1,
#     'lambda_l2': 0.2,
#     'max_depth': 15,
#     'objective': 'multiclass',  # 目标函数
#     'num_class': 3,
#     'num_leaves':40
# }
# validation data
# accuracy_score:  0.6191813643163802
# ---------------------------------------------------------------------------------------------------------
# precision_score(micro):  0.6191813643163802
# precision_score(macro):  0.609832357815999
# ---------------------------------------------------------------------------------------------------------
# f1_score(micro):  0.6191813643163802
# f1_score(macro):  0.5887113764830064
# ---------------------------------------------------------------------------------------------------------
# recall_score(micro):  0.6191813643163802
# recall_score(macro):  0.5748318566330418

# testing data
# accuracy_score:  0.5986018912585872
# ---------------------------------------------------------------------------------------------------------
# precision_score(micro):  0.5986018912585872
# precision_score(macro):  0.5904419306547104
# ---------------------------------------------------------------------------------------------------------
# f1_score(micro):  0.5986018912585872
# f1_score(macro):  0.5591086156813923
# ---------------------------------------------------------------------------------------------------------
# recall_score(micro):  0.5986018912585872
# recall_score(macro):  0.5420959144299468


## Clear data


正相關
 (1, 12), (0, 24), (7, 19), (7, 20), (10, 22) (10, 23) (11, 23) (50, 6) (50, 7) (50, 8) (50, 19) (50, 20) (51, 9) (51, 10) (51, 11)

12, 24, 7, 23, 41, 44, 45, 47, 50, 51, 62, 64, 65

負相關
(2, 3), (4, 5),

In [6]:
df_train_data_2 = pd.DataFrame(train_data)
df_train_labels_2 = pd.DataFrame(train_labels)
df_train_labels_2.rename(columns = {0:'Label'}, inplace = True)
df_train_data_2

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,0.853843,-0.709693,0.142649,-0.176952,0.117582,-0.152395,1.207672,0.245734,1.924513,0.601621,...,0.0,0.000000,0.255428,3.080873,0.255428,-1.787995,0.201544,0.352000,0.393182,0.055556
1,1.128771,-0.029401,0.142649,-0.176952,0.117582,-0.152395,1.001865,1.037051,0.440938,0.447094,...,0.0,0.000000,0.255428,3.080873,0.255428,-1.787995,0.201544,0.352000,0.393182,0.055556
2,0.760572,-0.029401,0.142649,-0.176952,0.117582,-0.152395,0.433292,0.068885,0.908483,1.140838,...,0.0,0.000000,0.255428,3.080873,0.255428,-1.787995,0.201544,0.352000,0.393182,0.055556
3,1.114048,-0.029401,0.142649,-0.172883,0.117582,-0.152395,0.060256,0.198770,0.098029,0.820122,...,0.0,0.000000,0.255428,3.080873,0.255428,-1.787995,0.201544,0.352000,0.393182,0.055556
4,0.448862,-0.029401,0.142649,-0.176952,0.117582,-0.152395,1.321162,0.342947,1.146333,1.163741,...,0.0,0.000000,0.255428,3.080873,0.255428,-1.787995,0.201544,0.352000,0.393182,0.055556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1174456,-0.129721,-0.786425,-0.241619,0.069267,-0.175944,0.029027,0.754470,-0.211944,0.696303,-1.608424,...,1.0,-0.124224,-0.496894,2.496879,0.000000,-9.937888,3.961347,0.237263,0.355828,0.031489
1174457,1.884997,0.018018,-0.241619,0.069267,1.691313,-1.663993,0.754470,0.379784,1.729425,-0.464348,...,1.0,-0.124224,-0.496894,2.496879,0.000000,-9.937888,3.961347,0.237263,0.355828,0.031489
1174458,0.328971,-0.786425,-0.241619,0.069267,-0.175944,0.029027,-1.562018,-0.681625,0.329797,1.898219,...,1.0,-0.124224,-0.496894,2.496879,0.000000,-9.937888,3.961347,0.237263,0.355828,0.031489
1174459,0.328971,1.541642,-0.241619,0.069267,-0.175944,0.029027,-1.562018,-1.484551,-0.186764,1.211692,...,1.0,-0.124224,-0.496894,2.496879,0.000000,-9.937888,3.961347,0.237263,0.355828,0.031489


將高度相關的刪除

In [7]:
df_train_data_2 = df_train_data_2.drop([65], axis=1)
df_train_data_2 = df_train_data_2.drop([64], axis=1)
df_train_data_2 = df_train_data_2.drop([62], axis=1)
df_train_data_2 = df_train_data_2.drop([51], axis=1)
# df_train_data_2 = df_train_data_2.drop([50], axis=1)
df_train_data_2 = df_train_data_2.drop([47], axis=1)
# df_train_data_2 = df_train_data_2.drop([45], axis=1)
df_train_data_2 = df_train_data_2.drop([44], axis=1)
df_train_data_2 = df_train_data_2.drop([41], axis=1)
# df_train_data_2 = df_train_data_2.drop([24], axis=1)
# df_train_data_2 = df_train_data_2.drop([23], axis=1)
df_train_data_2 = df_train_data_2.drop([12], axis=1)
# df_train_data_2 = df_train_data_2.drop([7], axis=1)


In [8]:
df_train_data_2

,0,1,2,3,4,5,6,7,8,9,...,57,58,59,60,61,63,66,67,68,69
0,0.853843,-0.709693,0.142649,-0.176952,0.117582,-0.152395,1.207672,0.245734,1.924513,0.601621,...,0.0,0.0,0.0,0.0,0.000000,3.080873,0.201544,0.352000,0.393182,0.055556
1,1.128771,-0.029401,0.142649,-0.176952,0.117582,-0.152395,1.001865,1.037051,0.440938,0.447094,...,0.0,1.0,0.0,0.0,0.000000,3.080873,0.201544,0.352000,0.393182,0.055556
2,0.760572,-0.029401,0.142649,-0.176952,0.117582,-0.152395,0.433292,0.068885,0.908483,1.140838,...,0.0,1.0,0.0,0.0,0.000000,3.080873,0.201544,0.352000,0.393182,0.055556
3,1.114048,-0.029401,0.142649,-0.172883,0.117582,-0.152395,0.060256,0.198770,0.098029,0.820122,...,0.0,0.0,1.0,0.0,0.000000,3.080873,0.201544,0.352000,0.393182,0.055556
4,0.448862,-0.029401,0.142649,-0.176952,0.117582,-0.152395,1.321162,0.342947,1.146333,1.163741,...,0.0,1.0,0.0,0.0,0.000000,3.080873,0.201544,0.352000,0.393182,0.055556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1174456,-0.129721,-0.786425,-0.241619,0.069267,-0.175944,0.029027,0.754470,-0.211944,0.696303,-1.608424,...,0.0,1.0,0.0,1.0,-0.124224,2.496879,3.961347,0.237263,0.355828,0.031489
1174457,1.884997,0.018018,-0.241619,0.069267,1.691313,-1.663993,0.754470,0.379784,1.729425,-0.464348,...,0.0,1.0,0.0,1.0,-0.124224,2.496879,3.961347,0.237263,0.355828,0.031489
1174458,0.328971,-0.786425,-0.241619,0.069267,-0.175944,0.029027,-1.562018,-0.681625,0.329797,1.898219,...,0.0,1.0,0.0,1.0,-0.124224,2.496879,3.961347,0.237263,0.355828,0.031489
1174459,0.328971,1.541642,-0.241619,0.069267,-0.175944,0.029027,-1.562018,-1.484551,-0.186764,1.211692,...,0.0,1.0,0.0,1.0,-0.124224,2.496879,3.961347,0.237263,0.355828,0.031489


In [9]:
df_eval_data_2 = pd.DataFrame(eval_data)
df_eval_labels_2 = pd.DataFrame(eval_labels)

In [10]:
df_eval_data_2 = df_eval_data_2.drop([65], axis=1)
df_eval_data_2 = df_eval_data_2.drop([64], axis=1)
df_eval_data_2 = df_eval_data_2.drop([62], axis=1)
df_eval_data_2 = df_eval_data_2.drop([51], axis=1)
# df_eval_data_2 = df_eval_data_2.drop([50], axis=1)
df_eval_data_2 = df_eval_data_2.drop([47], axis=1)
# df_eval_data_2 = df_eval_data_2.drop([45], axis=1)
df_eval_data_2 = df_eval_data_2.drop([44], axis=1)
df_eval_data_2 = df_eval_data_2.drop([41], axis=1)
# df_eval_data_2 = df_eval_data_2.drop([24], axis=1)
# df_eval_data_2 = df_eval_data_2.drop([23], axis=1)
df_eval_data_2 = df_eval_data_2.drop([12], axis=1)
# df_eval_data_2 = df_eval_data_2.drop([7], axis=1)

In [11]:
df_eval_data_2

,0,1,2,3,4,5,6,7,8,9,...,57,58,59,60,61,63,66,67,68,69
0,-0.163385,1.728609,0.142649,-0.176952,0.117582,-0.152395,0.367092,0.396664,-0.568763,-0.659308,...,0.0,0.0,0.0,0.0,0.000000,1.960784,0.336591,0.264297,0.315789,0.026261
1,-0.055390,1.519337,0.142649,-0.176952,0.117582,-0.152395,0.625994,-0.385464,-1.628485,0.106094,...,0.0,1.0,0.0,0.0,0.000000,1.960784,0.336591,0.264297,0.315789,0.026261
2,-0.163385,1.519337,0.142649,-0.176952,0.117582,-0.152395,0.423988,0.171544,-0.373376,0.901865,...,0.0,1.0,0.0,0.0,0.000000,1.960784,0.336591,0.264297,0.315789,0.026261
3,0.096820,-0.029401,0.142649,-0.176952,0.117582,-0.152395,0.900423,-0.511266,0.159667,-0.132701,...,0.0,1.0,0.0,0.0,0.000000,1.960784,0.336591,0.264297,0.315789,0.026261
4,0.096820,1.150617,0.142649,-0.176952,0.117582,-0.152395,-2.559886,0.032492,-0.380696,0.576335,...,0.0,0.0,1.0,0.0,0.000000,1.960784,0.336591,0.264297,0.315789,0.026261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1175297,-1.003403,1.080189,0.132555,-0.162858,0.133346,-0.165023,0.332277,-0.141980,-1.691026,0.816738,...,0.0,1.0,0.0,0.0,-0.554529,4.136691,0.123831,0.243421,0.363363,0.020290
1175298,-1.208349,0.028363,0.132555,-0.162858,0.133346,-0.165023,0.332277,-1.206222,-1.121051,-0.289549,...,0.0,1.0,0.0,0.0,-0.554529,4.136691,0.123831,0.243421,0.363363,0.020290
1175299,-1.078144,0.028363,0.132555,-0.162858,0.133346,-0.165023,-0.502565,-0.688454,-1.423030,0.056849,...,0.0,1.0,1.0,0.0,-0.554529,4.136691,0.123831,0.243421,0.363363,0.020290
1175300,-0.445393,0.028363,0.132555,-0.162858,0.133346,-0.165023,0.300544,-1.076358,-0.998571,0.375565,...,0.0,1.0,0.0,0.0,-0.554529,4.136691,0.123831,0.243421,0.363363,0.020290


In [12]:
frames_2 = [df_train_data_2, df_train_labels]

result_2 = pd.concat(frames_2, axis=1, join='inner')

# result

In [13]:
result_shuffle_2 = result_2.sample(frac = 1)

# result_shuffle

In [14]:
traing_data_2 = result_shuffle_2[:][0:100000]

validation_data_2= result_shuffle_2[:][100000:]

In [15]:
training_target_2 = traing_data_2['Label']
training_target_2 = training_target_2.to_numpy()
print(training_target_2, len(training_target_2))

print('-----------------------------------------------------')

validation_target_2 = validation_data_2['Label']
validation_target_2 = validation_target_2.to_numpy()
print(validation_target_2, len(validation_target_2))

[2 0 1 ... 0 1 0] 100000
-----------------------------------------------------
[2 0 0 ... 0 1 0] 1074461


In [16]:
traing_data_2=traing_data_2.drop(columns=['Label'])
training_feature_2 = traing_data_2.to_numpy()
print(training_feature_2, len(training_feature_2))

print('-----------------------------------------------------')

validation_data_2=validation_data_2.drop(columns=['Label'])
validation_feature_2 = validation_data_2.to_numpy()
print(validation_feature_2, len(validation_feature_2))


[[-0.66596339 -0.02291651  0.08483211 ...  0.25151057  0.31531532
   0.0332997 ]
 [-0.23969034  0.01516413  0.13047317 ...  0.35019973  0.65589354
   0.31864754]
 [ 1.25699912 -0.82572362  0.11409532 ...  0.2552      0.27899687
   0.02685285]
 ...
 [ 0.72968752  0.00299937  0.11816524 ...  0.23726346  0.35582822
   0.03148855]
 [-0.25020071  1.44543166  0.12886284 ...  0.25859492  0.36705202
   0.03225806]
 [-0.38201402  0.0243277   0.16573692 ...  0.25390625  0.34461538
   0.02722513]] 100000
-----------------------------------------------------
[[-1.9817566  -0.03174144  0.10932508 ...  0.21541787  0.33444816
   0.03581267]
 [ 0.29807988  2.4218313   0.06972306 ...  0.25034674  0.41551247
   0.03515264]
 [ 0.4565188  -1.70337072  0.1307201  ...  0.24638844  0.32899023
   0.01916933]
 ...
 [-1.22639964 -0.00897121  0.16407717 ...  0.30340333  0.36038186
   0.05405405]
 [ 1.13944343 -0.00881672  0.16990286 ...  0.25613276  0.42535211
   0.0514064 ]
 [-0.89959811  0.91995607  0.13564577

In [17]:
eval_data_2 = df_eval_data_2.to_numpy()
eval_data_2

array([[-0.16338479,  1.7286092 ,  0.14264851, ...,  0.26429675,
         0.31578947,  0.0262605 ],
       [-0.05539012,  1.51933705,  0.14264851, ...,  0.26429675,
         0.31578947,  0.0262605 ],
       [-0.16338479,  1.51933705,  0.14264851, ...,  0.26429675,
         0.31578947,  0.0262605 ],
       ...,
       [-1.07814393,  0.0283631 ,  0.13255495, ...,  0.24342105,
         0.36336336,  0.02028986],
       [-0.44539324,  0.0283631 ,  0.13255495, ...,  0.24342105,
         0.36336336,  0.02028986],
       [-1.55934112,  0.23100588,  0.13255495, ...,  0.24342105,
         0.36336336,  0.02028986]])

In [18]:
# 转换为Dataset数据格式
train = lgb.Dataset(training_feature_2, label=training_target_2)
validation = lgb.Dataset(validation_feature_2, label=validation_target_2)

# 参数
params = {
    'learning_rate': 0.1,
    'lambda_l1': 0.1,
    'lambda_l2': 0.2,
    'max_depth': 50,
    'objective': 'multiclass',  # 目标函数
    'num_class': 3,
    'num_leaves':50,
    "boosting":'dart'
}

# 模型训练
gbm = lgb.train(params, train, valid_sets=[validation])


# 模型预测
y_pred = gbm.predict(validation_feature_2)
y_pred = [list(x).index(max(x)) for x in y_pred]
print(y_pred)

# 模型评估
print('validation data')
print("accuracy_score: ", accuracy_score(validation_target_2, y_pred))

print('---------------------------------------------------------------------------------------------------------')
print("precision_score(micro): ", precision_score(validation_target_2, y_pred, average='micro'))
print("precision_score(macro): ", precision_score(validation_target_2, y_pred, average='macro'))


print('---------------------------------------------------------------------------------------------------------')
print("f1_score(micro): ", f1_score(validation_target_2, y_pred, average='micro'))
print("f1_score(macro): ", f1_score(validation_target_2, y_pred, average='macro'))

print('---------------------------------------------------------------------------------------------------------')
print("recall_score(micro): ", recall_score(validation_target_2, y_pred, average='micro'))
print("recall_score(macro): ", recall_score(validation_target_2, y_pred, average='macro'))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.079199 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14214
[LightGBM] [Info] Number of data points in the train set: 100000, number of used features: 62
[LightGBM] [Info] Start training from score -0.610296
[LightGBM] [Info] Start training from score -1.517725
[LightGBM] [Info] Start training from score -1.437167


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



accuracy_score:  0.6060657390077443
---------------------------------------------------------------------------------------------------------
precision_score(micro):  0.6060657390077443
precision_score(macro):  0.5977715470381271
---------------------------------------------------------------------------------------------------------
f1_score(micro):  0.6060657390077443
f1_score(macro):  0.5663701726952867
---------------------------------------------------------------------------------------------------------
recall_score(micro):  0.6060657390077443
recall_score(macro):  0.5491522289487432


In [19]:
# testing
# 模型预测
y_pred = gbm.predict(eval_data_2)
y_pred = [list(x).index(max(x)) for x in y_pred]
print(y_pred)

# 模型评估
print('testing data')
print("accuracy_score: ", accuracy_score(eval_labels, y_pred))

print('---------------------------------------------------------------------------------------------------------')
print("precision_score(micro): ", precision_score(eval_labels, y_pred, average='micro'))
print("precision_score(macro): ", precision_score(eval_labels, y_pred, average='macro'))


print('---------------------------------------------------------------------------------------------------------')
print("f1_score(micro): ", f1_score(eval_labels, y_pred, average='micro'))
print("f1_score(macro): ", f1_score(eval_labels, y_pred, average='macro'))


print('---------------------------------------------------------------------------------------------------------')
print("recall_score(micro): ", recall_score(eval_labels, y_pred, average='micro'))
print("recall_score(macro): ", recall_score(eval_labels, y_pred, average='macro'))


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



accuracy_score:  0.5936950673103594
---------------------------------------------------------------------------------------------------------
precision_score(micro):  0.5936950673103594
precision_score(macro):  0.5841168267093452
---------------------------------------------------------------------------------------------------------
f1_score(micro):  0.5936950673103594
f1_score(macro):  0.5551242615301862
---------------------------------------------------------------------------------------------------------
recall_score(micro):  0.5936950673103594
recall_score(macro):  0.5389353652595765
